# Análisis exploratorio y limpieza de datos

In [1]:
import pandas as pd

## Tabla Ventas

In [16]:
dfv = pd.read_excel('./raw data/Tabla Ventas.xlsx', header=2, usecols=['NroFactura', 'FechaFactura','Cliente','MontoSinImp','Impuestos','MontoFactura'])

In [17]:
dfv

,NroFactura,FechaFactura,Cliente,MontoSinImp,Impuestos,MontoFactura
0,1,2014-01-03,Semillas Santo Mongo,765.68,203.53,969.21
1,2,2014-01-03,Buffalo Cnia,420.60,111.80,532.40
2,3,2014-01-03,Englewood Cnia,108.98,28.96,137.94
3,4,2014-01-04,"Ross, Christian",304.94,81.05,385.99
4,5,2014-01-04,"Diaz, Gabriel",107.07,28.45,135.52
...,...,...,...,...,...,...
4395,4396,2016-07-28,"Ward, Jeremy",113.00,23.73,136.73
4396,4397,2016-07-28,Coffs Harbour Cnia,409.00,85.89,494.89
4397,4398,2016-07-28,"Miller, Seth",801.00,168.21,969.21
4398,4399,2016-07-28,"Sharma, Ethan",856.00,179.76,1035.76


In [18]:
dfv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4400 entries, 0 to 4399
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   NroFactura    4400 non-null   int64         
 1   FechaFactura  4400 non-null   datetime64[ns]
 2   Cliente       4400 non-null   object        
 3   MontoSinImp   4400 non-null   float64       
 4   Impuestos     4400 non-null   float64       
 5   MontoFactura  4400 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(1)
memory usage: 206.4+ KB


* Nulos

In [20]:
dfv.isna().sum()

NroFactura      0
FechaFactura    0
Cliente         0
MontoSinImp     0
Impuestos       0
MontoFactura    0
dtype: int64

* Duplicados

In [22]:
dfv.duplicated().sum()

0

In [24]:
dfv.duplicated(subset=['NroFactura'],keep=False).sum()

0

## Tabla Clientes

In [106]:
dfc = pd.read_csv('./raw data/Tabla Clientes.csv', sep=";" , header=2, encoding='latin',usecols=['Cliente','Tipo','Teléfono','Ubicación'])

In [107]:
dfc

,Cliente,Tipo,Teléfono,Ubicación
0,"Adams, Fernando",Persona,373846615,"Miami, Estados Unidos"
1,"Adams, Wyatt",Persona,646775929,"Madrid, España"
2,"Alonso, Julio",Persona,667028591,"Buenos Aires, Argentina"
3,"Ana, Maria",Persona,707344341,"Córdoba, Argentina"
4,"Anderson, Nicholas",Persona,743415512,"La Paz, Bolivia"
...,...,...,...,...
102,Wapato Cnia,Empresa,501853861,"Santa Cruz de la Sierra, Bolivia"
103,"Ward, Jeremy",Persona,281797334,"Córdoba, Argentina"
104,"Washington, Jack",Persona,490574837,"Río de Janeiro, Brasil"
105,"Washington, Jose",Persona,342142162,"San Juan, Puerto Rico"


In [44]:
dfc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Cliente    107 non-null    object
 1   Tipo       107 non-null    object
 2   Teléfono   107 non-null    int64 
 3   Ubicación  107 non-null    object
dtypes: int64(1), object(3)
memory usage: 3.5+ KB


* Nulos

In [45]:
dfc.isna().sum()

Cliente      0
Tipo         0
Teléfono     0
Ubicación    0
dtype: int64

* Duplicados

In [46]:
dfc.duplicated().sum()

0

In [47]:
dfc.duplicated(subset=['Cliente']).sum()

0

No hay clientes duplicados

## Normalización
De a cuerdo a lo evaluado en las tablas cliente y ventas, se deben realizar las siguientes transformaciones de normalización
* Tabla clientes:
    - Se debe asignar un id único a cada cliente (ya se verificó que son únicos)
    - Se debe crear una tabla tipo de persona con un id unico y reemplazar la columna "Tipo" en la tabla clientes por su correspondiente id
    - Hay que normalizar las localizaciones
* Tabla Ventas:
    - Asignar el id de cada cliente en la tabla ventas y eliminar el campo clientes

### Tipo persona

In [108]:
dfc['Tipo'].value_counts()

Persona    65
Empresa    42
Name: Tipo, dtype: int64

In [57]:
tipos_persona = list(set(dfc['Tipo'].values))

In [62]:
dfp = pd.DataFrame({"IdPersona":[1,2],"Tipo": tipos_persona})

In [63]:
dfp

,IdPersona,Tipo
0,1,Empresa
1,2,Persona


### IdCliente y IdPersona en tabla clientes

In [109]:
dfc = dfc.merge(dfp,on=['Tipo'])

In [110]:
dfc['IdCliente'] = [i for i in range(1,len(dfc)+1)]

In [111]:
dfc = dfc[['IdCliente','Cliente','IdPersona','Teléfono','Ubicación']]

In [112]:
dfc

,IdCliente,Cliente,IdPersona,Teléfono,Ubicación
0,1,"Adams, Fernando",2,373846615,"Miami, Estados Unidos"
1,2,"Adams, Wyatt",2,646775929,"Madrid, España"
2,3,"Alonso, Julio",2,667028591,"Buenos Aires, Argentina"
3,4,"Ana, Maria",2,707344341,"Córdoba, Argentina"
4,5,"Anderson, Nicholas",2,743415512,"La Paz, Bolivia"
...,...,...,...,...,...
102,103,Transportes Pampa,1,397375640,"Asunción, Paraguay"
103,104,Trenton Cnia,1,805211764,"San José, Costa Rica"
104,105,Victoria Cnia,1,694310799,"Maracaibo, Venezuela"
105,106,Viga Ingenieros,1,255869094,"Madrid, España"


### Geolocalización - normalización

In [113]:
dfc['Ubicación'].duplicated().sum()

67

In [117]:
dfciudad = dfc[['Ubicación']].drop_duplicates(keep='first').sort_values(by=['Ubicación'])

In [118]:
dfciudad['IdCiudad'] = [i for i in range(1, len(dfciudad)+1)]

In [119]:
dfciudad

,Ubicación,IdCiudad
60,"Asunción, Paraguay",1
11,"Barcelona, España",2
26,"Barranquilla, Colombia",3
41,"Bogotá, Colombia",4
2,"Buenos Aires, Argentina",5
10,"Caracas, Venezuela",6
8,"Ciudad de Guatemala, Guatemala",7
28,"Ciudad de México, México",8
3,"Córdoba, Argentina",9
16,"Guadalajara, México",10


* Encuentro latitud y longitud con geocoder

In [94]:
import geocoder
# Funciones de latitud y longitud Geocoder
def find_latitud_geocoder(localizador):
    loc = geocoder.osm(localizador)
    return loc.lat

def find_longitud_geocoder(localizador):
    loc = geocoder.osm(localizador)
    return loc.lng

In [120]:
dfciudad['latitud'] = dfciudad['Ubicación'].map(find_latitud_geocoder)
dfciudad['longitud'] = dfciudad['Ubicación'].map(find_longitud_geocoder)

In [121]:
dfciudad[['Ciudad','País']] = dfciudad['Ubicación'].str.split(', ',expand=True)

In [122]:
dfciudad = dfciudad[['IdCiudad', 'Ciudad', 'País','latitud','longitud','Ubicación']]
dfciudad

,IdCiudad,Ciudad,País,latitud,longitud,Ubicación
60,1,Asunción,Paraguay,-25.280046,-57.634381,"Asunción, Paraguay"
11,2,Barcelona,España,41.382894,2.177432,"Barcelona, España"
26,3,Barranquilla,Colombia,10.993870,-74.792718,"Barranquilla, Colombia"
41,4,Bogotá,Colombia,4.652954,-74.083564,"Bogotá, Colombia"
2,5,Buenos Aires,Argentina,-34.607568,-58.437089,"Buenos Aires, Argentina"
10,6,Caracas,Venezuela,10.506093,-66.914601,"Caracas, Venezuela"
8,7,Ciudad de Guatemala,Guatemala,14.622233,-90.518519,"Ciudad de Guatemala, Guatemala"
28,8,Ciudad de México,México,19.432630,-99.133178,"Ciudad de México, México"
3,9,Córdoba,Argentina,-31.416687,-64.183419,"Córdoba, Argentina"
16,10,Guadalajara,México,20.672037,-103.338396,"Guadalajara, México"


* Mergeando con tabla clientes

In [123]:
dfc =dfc.merge(dfciudad[['IdCiudad','Ubicación']], on=['Ubicación'], how='left')


In [124]:
dfc.drop(columns=['Ubicación'], inplace=True)
dfc

,IdCliente,Cliente,IdPersona,Teléfono,IdCiudad
0,1,"Adams, Fernando",2,373846615,20
1,2,"Adams, Wyatt",2,646775929,16
2,3,"Alonso, Julio",2,667028591,5
3,4,"Ana, Maria",2,707344341,9
4,5,"Anderson, Nicholas",2,743415512,13
...,...,...,...,...,...
102,103,Transportes Pampa,1,397375640,1
103,104,Trenton Cnia,1,805211764,27
104,105,Victoria Cnia,1,694310799,18
105,106,Viga Ingenieros,1,255869094,16


### IdCliente en tabla ventas

In [ ]:
dfv = dfv.merge(dfc[['IdCliente','Cliente']], on=['Cliente'], how='left')

In [78]:
dfv.isna().sum()

NroFactura      0
FechaFactura    0
Cliente         0
MontoSinImp     0
Impuestos       0
MontoFactura    0
IdCliente       0
dtype: int64

In [126]:
dfv

,NroFactura,FechaFactura,IdCliente,MontoSinImp,Impuestos,MontoFactura
0,1,2014-01-03,100,765.68,203.53,969.21
1,2,2014-01-03,67,420.60,111.80,532.40
2,3,2014-01-03,77,108.98,28.96,137.94
3,4,2014-01-04,54,304.94,81.05,385.99
4,5,2014-01-04,14,107.07,28.45,135.52
...,...,...,...,...,...,...
4395,4396,2016-07-28,62,113.00,23.73,136.73
4396,4397,2016-07-28,73,409.00,85.89,494.89
4397,4398,2016-07-28,39,801.00,168.21,969.21
4398,4399,2016-07-28,57,856.00,179.76,1035.76


In [80]:
dfv = dfv[['NroFactura','FechaFactura','IdCliente','MontoSinImp','Impuestos','MontoFactura']]

In [127]:
dfv

,NroFactura,FechaFactura,IdCliente,MontoSinImp,Impuestos,MontoFactura
0,1,2014-01-03,100,765.68,203.53,969.21
1,2,2014-01-03,67,420.60,111.80,532.40
2,3,2014-01-03,77,108.98,28.96,137.94
3,4,2014-01-04,54,304.94,81.05,385.99
4,5,2014-01-04,14,107.07,28.45,135.52
...,...,...,...,...,...,...
4395,4396,2016-07-28,62,113.00,23.73,136.73
4396,4397,2016-07-28,73,409.00,85.89,494.89
4397,4398,2016-07-28,39,801.00,168.21,969.21
4398,4399,2016-07-28,57,856.00,179.76,1035.76


## Exportación de data limpia y normalizada

In [131]:
dfv.to_csv('./clean data/Ventas.csv',sep=";",index=False,float_format='%.2f',encoding='utf-8')
dfc.to_csv('./clean data/Clientes.csv',sep=";",index=False,encoding='latin')
dfp.to_csv('./clean data/Personas.csv',index=False,sep=";",encoding='utf-8')
dfciudad.to_csv('./clean data/Ciudad.csv',index=False,sep=";",encoding='latin')